# Imports

In [209]:
# Used to pull data from Reddit
import praw
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import statsmodels.formula.api


from gensim.models import Word2Vec
from nltk.corpus import brown
import nltk
import tensorflow as tf
import tensorflow_hub as hub
import multiprocessing
from google.cloud import bigquery

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#nltk.download('punkt')
#nltk.download('brown')

In [2]:
sql_query = '''
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2018_10` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2018_11` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2018_12` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2019_01` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2019_02` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2019_03` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
'''

# Put query results into df
comment_df = pd.read_gbq(sql_query,
                         project_id='w266-240122',
                         dialect='standard')

# Convert date into proper date/time
comment_df['created_dt_tm'] = comment_df['created_utc'].apply(lambda x: dt.datetime.fromtimestamp(x))

# Create field for month
comment_df['created_dt_month'] = comment_df['created_dt_tm'].dt.to_period('M').dt.to_timestamp()

In [3]:
sql_query = """
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2018_10` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2018_11` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2018_12` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2019_01` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2019_02` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2019_03` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
"""

post_df = pd.read_gbq(sql_query,
                      project_id='w266-240122',
                      dialect='standard')

# Convert date into proper date/time
post_df['created_dt_tm'] = post_df['created_utc'].apply(lambda x: dt.datetime.fromtimestamp(x))

# Create field for month
post_df['created_dt_month'] = post_df['created_dt_tm'].dt.to_period('M').dt.to_timestamp()

# Functions

In [150]:
def preprocess_text(comment_df, post_df, subreddit, lower=False):
    
    # TODO remove symbols, contractions
    # TODO lowercase words?
    # TODO Treat numbers as something else
    # Add beginning and end of sentence?
    # TODO how do we deal with stop words?
    
    # Body text from post dataframe
    post_text = list(post_df[(post_df['is_self'] == True) &
                             (~post_df['selftext'].isin(['[removed]', '[deleted]'])) &
                             post_df['subreddit'].isin(subreddit)]['selftext'].unique())
    
    # Add in text from title
    post_text += list(post_df[post_df['is_self'] == True]['title'].unique())
    
    # Add in text from post comments
    post_text += list(comment_df[(comment_df['subreddit'].isin(subreddit)) & 
                                     (~comment_df['body'].isin(['[removed]', '[deleted]']))]['body'])
    
    # Put all text into dataframe and drop dupes
    text_df = pd.DataFrame(post_text, columns=['text'])
    text_df.drop_duplicates(inplace=True)
    
    # Tokenize at sentence level
    text_df['sent_tokenized'] = text_df['text'].apply(nltk.sent_tokenize)
    text_df['sent_count'] = text_df['sent_tokenized'].apply(lambda x: len(x))

    # Put tokenized body into a list
    sent_list = list(text_df[text_df['sent_count'] > 0]['sent_tokenized'].apply(pd.Series).stack().unique())

    # Put list into dataframe
    sent_df = pd.DataFrame(sent_list, columns=['sentence'])
    
    # lowercase
    if lower == True:
        sent_df['sentence'] = sent_df['sentence'].str.lower()

    # Tokenize each sentence at the word level
    sent_df['word_token'] = sent_df['sentence'].apply(nltk.word_tokenize)
    
    return sent_df

In [151]:
def train_embedding(sent_df):
    
    # train word embedding
    embedding = Word2Vec(list(sent_df['word_token']),
                         size=100,
                         window=5, 
                         min_count=5, 
                         negative=15, 
                         iter=10, workers=6)
    
    return embedding

# Basic EDA

In [6]:
comment_df['created_dt_month'].value_counts()

2019-03-01    109492
2018-10-01    107297
2019-01-01    100792
2018-11-01     91601
2019-02-01     88902
2018-12-01     73725
2018-09-01       765
Name: created_dt_month, dtype: int64

In [7]:
post_df['created_dt_month'].value_counts()

2018-10-01    11185
2019-01-01    10551
2019-03-01    10473
2018-11-01     9703
2019-02-01     9316
2018-12-01     7986
2018-09-01       94
Name: created_dt_month, dtype: int64

In [8]:
sent_df = preprocess_text(comment_df, 
                          post_df,
                          ['democrats', 'progressive'])

In [9]:
sent_df['sent_length'] = sent_df['word_token'].apply(lambda x: len(x))

# Emebddings from Democratic Leaning Subreddits

### Text PreProcessing

In [208]:
# Empty list to store the embeddings
embedding_list = []

# Dates to iterate over
dates = comment_df.created_dt_month.sort_values().unique()[1:]

# Iterate for each month and create embedding for it
for d in dates:
    sent_df = preprocess_text(comment_df[comment_df['created_dt_month'] == d], 
                              post_df[post_df['created_dt_month'] == d],
                              ['democrats', 'progressive'], 
                              lower=False)
    word_embedding = train_embedding(sent_df)
    embedding_list.append(word_embedding)

# Sentiment Model

In [17]:
def load_lexicon(filename):
    """
    Load a file from Bing Liu's sentiment lexicon
    (https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html), containing
    English words in Latin-1 encoding.
    
    One file contains a list of positive words, and the other contains
    a list of negative words. The files contain comment lines starting
    with ';' and blank lines, which should be skipped.
    """
    lexicon = []
    with open(filename, encoding='latin-1') as infile:
        for line in infile:
            line = line.rstrip()
            if line and not line.startswith(';'):
                lexicon.append(line)
    return lexicon

pos_words = load_lexicon('positive-words.txt')
neg_words = load_lexicon('negative-words.txt')

In [210]:
embedding_df = pd.DataFrame(embedding_list[0].wv.vectors)
embedding_df.index = embedding_list[0].wv.index2word

In [211]:
pos_vectors = embedding_df.reindex(index=pos_words).dropna()
neg_vectors = embedding_df.reindex(index=neg_words).dropna()

vectors = pd.concat([pos_vectors, neg_vectors])
targets = np.array([1 for entry in pos_vectors.index] + [-1 for entry in neg_vectors.index])
labels = list(pos_vectors.index) + list(neg_vectors.index)

In [212]:
train_vectors, test_vectors, train_targets, test_targets, train_labels, test_labels = \
train_test_split(vectors, targets, labels, test_size=0.1, random_state=0)

In [213]:
model = SGDClassifier(loss='log', random_state=0, n_iter=100)
model.fit(train_vectors, train_targets)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=100, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=0, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [214]:
accuracy_score(model.predict(test_vectors), test_targets)

0.7373737373737373

In [284]:
def vecs_to_sentiment(vecs):
    # predict_log_proba gives the log probability for each class
    predictions = model.predict_log_proba(vecs)

    # To see an overall positive vs. negative classification in one number,
    # we take the log probability of positive sentiment minus the log
    # probability of negative sentiment.
    return predictions[:, 1] - predictions[:, 0]


def words_to_sentiment(words):
    vecs = embedding_df.loc[words].dropna()
    log_odds = vecs_to_sentiment(vecs)
    return pd.DataFrame({'sentiment': log_odds}, index=vecs.index)


# Show 20 examples from the test set
words_to_sentiment(test_labels).iloc[:10]

,sentiment
murderous,-1.566623
horrific,-1.845333
trusted,-1.121001
murder,-2.698865
clear,1.940544
benefits,-2.822477
respect,-1.829895
booming,-1.401345
irrelevant,-1.251382
partisan,-0.333362


In [216]:
import re
TOKEN_RE = re.compile(r"\w.*?\b")
# The regex above finds tokens that start with a word-like character (\w), and continues
# matching characters (.+?) until the next word break (\b). It's a relatively simple
# expression that manages to extract something very much like words from text.


def text_to_sentiment(text):
    tokens = [token.casefold() for token in TOKEN_RE.findall(text)]
    sentiments = words_to_sentiment(tokens)
    return sentiments['sentiment'].mean()

In [281]:
text_to_sentiment('Trump')

-0.9871121835807091

In [282]:
text_to_sentiment('Obama')

-0.9871121835807091

In [275]:
text_to_sentiment('Trump')

-2.917751795003542

In [290]:
text_to_sentiment('Republicans')

0.6986753252329718

In [250]:
text_to_sentiment('Trump')

-2.917751795003542

### ELMo

In [20]:
# Download ELMo to a local directory
!mkdir module_elmo

# Download the module, and uncompress it to the destination folder. 
!curl -L "https://tfhub.dev/google/elmo/2?tf-hub-format=compressed" | tar -zxvC module_elmo

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  331M    0   415    0     0    369      0  10d 21h  0:00:01  10d 21h   369x assets/
x saved_model.pb
x tfhub_module.pb
x variables/
x variables/variables.index
100  331M  100  331M    0     0  2054k      0  0:02:45  0:02:45 --:--:-- 2379k 2614kk      0  0:02:59  0:01:26  0:01:33 2755k   0  2087k      0  0:02:42  0:01:43  0:00:59 3369k      0  0:02:41  0:01:51  0:00:50 2300k  0:01:56  0:00:44 2391k0:02:44  0:02:25  0:00:19 2082k



In [4]:
# Initialize elmo
elmo = hub.Module('module_elmo', trainable=True)
embeddings = elmo(['Trump'], signature="default", as_dict=True)['elmo']

# Get word embedding
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embeddings)

Instructions for updating:
Colocations handled automatically by placer.


W0715 22:15:39.981253 4808177088 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0715 22:15:40.683024 4808177088 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [11]:
message_embeddings[0][0]

1024

In [5]:
# Download ELMo to a local directory
!mkdir module_elmo

# Download the module, and uncom`press it to the destination folder. 
!curl -L "https://tfhub.dev/google/elmo/2?tf-hub-format=compressed" | tar -zxvC module_elmo

# Initialize elmo
elmo = hub.Module('module_elmo', trainable=True)
embeddings = elmo(['This is an example sentence that I want an embedding for'], signature="default", as_dict=True)['elmo']

# Get word embedding
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embeddings)

message_embeddings.shape

(1, 11, 1024)

# Questions
* Sentiment analysis labeled data
* How would we go about scoring the performance of our models?
    * I proposed to use the scores that are available on the reddit
* Should we train the sentiment classifier at a sentence or post level?
    * Each post may contain multiple sentences
